In [ ]:
import random

#Card Values for Blackjack based on French-suited playing cards
#Ace is counted as 11 if player has 2 cards. Ace is counted as 1 if more than 2 cards
suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10, 'Queen':10, 'King':10, 'Ace':11}

In [ ]:
class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit 
        self.rank = rank 
        self.value = values[rank] #Card Value in Blackjack
        
    def __str__(self): #return the card as a string
        return self.rank + ' of ' + self.suit

In [ ]:
class Deck:
        
    def __init__(self):
        #Create new deck of 52 cards
        self.deck = [] 
        for suit in suits: #('Hearts', 'Diamonds', 'Spades', 'Clubs')
            for rank in ranks: #(2 to Ace)
                self.deck.append(Card(suit,rank))
        #start the deck shuffled
        random.shuffle(self.deck)
        
    def deal_one(self):
        #Release one card from the top of the deck
        return self.deck.pop()


In [ ]:
class Player:
    
    def __init__(self, name, money):
        self.name = name
        self.bet = 0
        self.cards_on_hand = []
        self.stats = {
            "Win" : 0,
            "Loss" : 0,
            "Bust" : 0,
            "Wallet" : money
        }
        
    #display player stats
    def __str__(self):
        return "Player: " +self.name+ ", Wallet: " +str(self.stats["Wallet"])+ "\nWin: " +str(self.stats["Win"])+ ", Loss: " +str(self.stats["Loss"])+ ", Bust: " +str(self.stats["Bust"])
    
    #display cards on hand
    def display_cards(self):
        print(f"{self.name} has {str(len(self.cards_on_hand))} cards on hand:")
        for i in range(len(self.cards_on_hand)):
            print(f"[{i+1}] - {self.cards_on_hand[i]}") 
        print( f"Total Value: {str(self.deck_value())} \n" )
    
    #draw one card and add to the deck
    def draw_card(self, card):
        self.cards_on_hand.append(card)
        print(f"{self.name} draws a {card}\n")
        
    #draw 2 cards 
    def draw_hand(self, card_one, card_two):
        self.cards_on_hand.append(card_one)
        self.cards_on_hand.append(card_two)
        print(f"\n{self.name} draws a {card_one} and a {card_two}\n")
    
    #flush cards on hand
    def flush_hand(self):
        self.cards_on_hand = []
    
    #return total value of cards on hand
    def deck_value(self):
        deck_value = 0
        
        if len(self.cards_on_hand) > 2: 
            for cards in self.cards_on_hand:
                if cards.value == 11: #check for Ace
                    #Ace is counted as 11 if player has 2 cards. Ace is counted as 1 if more than 2 cards
                    deck_value += cards.value - 10
                else:
                    deck_value += cards.value
        else: #2 cards or less
            for cards in self.cards_on_hand:
                deck_value += cards.value           

        return deck_value
   

In [ ]:
##Banker/The House##
#Inherits from player with function variations
class Banker(Player):
    #draw 2 cards 
    def draw_hand(self, card_one, card_two):
        self.cards_on_hand.append(card_one)
        self.cards_on_hand.append(card_two)
        print(f"\n{self.name} draws two cards.\n")
    
    #display cards on hand with first card covered
    def display_cards_covered(self):
        print(f"{self.name} has {str(len(self.cards_on_hand))} cards on hand:")
        print(f"[1] - ##First Card Covered## ")
        for i in range(len(self.cards_on_hand)):
            if i == 0:
                continue
            print(f"[{i+1}] - {self.cards_on_hand[i]}")     

In [ ]:
###################
# BLACKJACK RULES #
###################
# 1. Player busts when > 21 pts
# 2. Player can only hold maximum of 5 cards
# 3. If player holds more than 2 cards AND have an Ace, the Ace is counted as 1 pt.
# 4. Player wins DOUBLE of bet amount if Blackjack with 2 cards.
# 5. Player wins TRIPLE of bet amount if both cards are Aces on first turn.
# 6. Player wins SEVEN times of bet amount if Blackjack with 3 cards (7-7-7)

def house_rules(player):
    
    #Blackjack and Double Ace
    if len(player.cards_on_hand) == 2 and player.deck_value() >= 21:
        if player.deck_value() == 22: #Double Ace
            return 'Double-Ace'
        else:
            return 'Blackjack'
    
    #Other win conditions 
    if player.deck_value() == 21: 
        if len(player.cards_on_hand) == 3 and player.cards_on_hand[0].value == 7:
            return 'Triple Seven' #7-7-7
        return 'Twenty-One'
    
    #Lose conditions
    if player.deck_value() > 21:
        return 'Bust'
    
    return 'Points'

#payout
def payout(player, bet):
    #win on 21 or banker busts
    payout = bet
    
    #special wins
    if house_rules(player) == 'Blackjack':
        payout = bet*2
    elif house_rules(player) == 'Double-Ace':
        payout = bet*3
    elif house_rules(player) == 'Triple Seven':
        payout = bet*7
        
    return payout

def player_wins(player, banker):
    player.stats["Win"] += 1
    player.stats['Wallet'] += payout(player, player.bet)
    banker.display_cards()
    print(f"{player.name} wins - {house_rules(player)}.\n")
    
def house_wins(player, banker):
    player.stats["Loss"] += 1
    player.stats['Wallet'] -= payout(banker, player.bet)
    banker.display_cards()
    print(f"{banker.name} wins - {house_rules(banker)}.\n")
    
def player_bust(player):
    player.stats["Bust"] += 1
    player.stats["Wallet"] -= player.bet
    print(f"{player.name} busts.\n")
    
def house_bust(player, banker):
    player.stats["Win"] += 1
    player.stats["Wallet"] += player.bet
    banker.display_cards()
    print(f"{banker.name} busts.\n")
    
def i_win_check(player, banker):
    #Blackjack and Double Ace Push
    if (house_rules(player) == 'Blackjack' and house_rules(banker) == 'Blackjack') or (house_rules(player) == 'Double-Ace' and house_rules(banker) == 'Double-Ace'):
        banker.display_cards()
        player.display_cards()
        print("\nPush!")   
        return True
    #Blackjack or Double-Ace, House Loses.
    elif house_rules(player) == 'Blackjack' or house_rules(player) == 'Double-Ace': 
        player_wins(player, banker)
        return True        
    #Blackjack or Double-Ace, Player Loses.
    elif house_rules(banker) == 'Blackjack' or house_rules(banker) == 'Double-Ace': 
        house_wins(player, banker)
        return True
    
    return False

In [ ]:
##################
# BLACKJACK GAME #
##################
def blackjack():
    #Initialize the Game
    player_options = {'H': 'Hit', #get another card
                      'S': 'Stand', #hold to cards
                      'F': 'Fold'} #fold cards, loses bet
    player_input = ''
    
    #Initialize Banker
    banker = Banker('The House', 0)
    #Initialize Player
    money = 100 #default money given is 100
    player = Player('Player One', money)
    
    while True: #loops until player quits
        #start with a new shuffled deck
        deck = Deck()
        #set game loop to true
        game_on = True
        #flush the decks for new play
        player.flush_hand()
        banker.flush_hand()
        
        print(player)
        try:
            player.bet  = int(input('Place your bet (5-100). (0) to Quit Game: '))
            
            if player.bet == 0:
                print("Game Ended.")
                break
            elif player.bet < 5 or player.bet > 100:
                print(f'Only bet between 5-100 is allowed')
                continue
            elif player.bet > player.stats['Wallet']:
                print("You may not bet more than what you have.")
                print(f"You have {str(player.stats['Wallet'])} in your wallet.")
                continue
            
            print(f"You bet {player.bet}")
        except ValueError:
            print(f'Input Error: NOT an integer')
            continue
        
        ##DEAL CARDS##
        banker.draw_hand(deck.deal_one(),deck.deal_one())
        banker.display_cards_covered()
        player.draw_hand(deck.deal_one(),deck.deal_one())
        player.display_cards()
        
        ##CHECK FOR INSTANT WINS##
        if i_win_check(player, banker):
            game_on = False
            
        ##CONTINUE GAME##
        while game_on:
            while player_input not in player_options:
                player_input  = str(input('[H]it | [S]tand | [F]old: ').capitalize())

            #FOLD - Fold game, loses bet money.
            if player_input == 'F':
                player.stats['Wallet'] -= player.bet
                print(f"{player.name} folds.")
                game_on = False

            #HIT - Draws one card, check for win/bust condition
            elif player_input == 'H':
                player.draw_card(deck.deal_one())
                player.display_cards()

                if house_rules(player) == 'Bust': #player busts
                    player_bust(player)
                    game_on = False

            #STAND - The house will play against player
            elif player_input == 'S':
                
                if player.deck_value() < 17 and len(player.cards_on_hand) < 5: #Deck Value must be > 16
                    print("Your deck value must be 17 pts or more")
                elif player.deck_value() == banker.deck_value(): #House Push
                    banker.display_cards()
                    print("\nPush!")
                    game_on = False
                elif player.deck_value() < banker.deck_value(): #House Wins
                    house_wins(player, banker)
                    game_on = False
                elif player.deck_value() > banker.deck_value(): #House is challenged
                    #stops looping: [1] Reached 5 cards [2] Bust [3] Deck Value > Player
                    while (player.deck_value() > banker.deck_value()) and len(banker.cards_on_hand) < 5:
                        banker.draw_card(deck.deal_one())

                        if house_rules(banker) == 'Bust': #banker busts
                            house_bust(player, banker)
                            game_on = False
                    
                    if player.deck_value() < banker.deck_value() and banker.deck_value() < 22: #banker wins on pts
                        house_wins(player, banker)
                        game_on = False
                    elif player.deck_value() > banker.deck_value() and player.deck_value() < 22: #player wins on pts:
                        player_wins(player, banker)
                        game_on = False
                    elif player.deck_value() == banker.deck_value(): #push
                        banker.display_cards()
                        player.display_cards()
                        print("\nPush!")   
                        game_on = False

            player_input = '' #flush the input             
    
